In [1]:
from CLS_ring import CLSforRing
from utils import load_and_resample, extract_sec, preprocess, forpred
import torch
import ring_voice
device = torch.device('cuda:1')
model = ring_voice.model
model.load_state_dict(torch.load("/home/ubuntu/jupyter/SKT/log/ring_voice_binary_2/model.pt"))
model.to(device)
model.eval()

# 코덱별 샘플링 레이트 및 주파수 범위 설정
codec_config = {
    'EVS': {'sample_rate': 16000, 'freq_range': (0, 8000)},
    'AMR-WB': {'sample_rate': 16000, 'freq_range': (0, 8000)},
    'AMR': {'sample_rate': 8000, 'freq_range': (0, 4000)},
}
selected_codec = 'EVS'
config = codec_config[selected_codec]
target_sr = config['sample_rate']
n_fft = 1024  # FFT 크기
hop_length = n_fft // 4  # 일반적으로 hop_length는 n_fft의 1/4로 설정
hann_window = torch.hann_window(n_fft)
        

Seed set to 2021
/tmp/ipykernel_593089/750085668.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/home/ubuntu/jupyter/SKT/log/ring_voic

In [2]:
import os
import numpy as np
test_sample1_path = "/home/ubuntu/jupyter/SKT/data/Scenario/music.wav"
test_sample2_path = "/home/ubuntu/jupyter/SKT/data/Scenario/system_o.wav" # PCM_30_wb
test_sample3_path = "/home/ubuntu/jupyter/SKT/data/Scenario/system_x1.wav" # PCM_15_wb

test_sample4_path = "/home/ubuntu/jupyter/SKT/data/Scenario/music+speech.wav"
test_sample5_path = "/home/ubuntu/jupyter/SKT/data/Scenario/music+speech(noise).wav"

test_sample6_path = "/home/ubuntu/jupyter/SKT/data/Scenario/system_o+speech.wav" # ANNC_1508_wb + start @ 60sec
test_sample7_path = "/home/ubuntu/jupyter/SKT/data/Scenario/system_o+speech(noise).wav" # INTL_TERM_ANM_wb + start @ 4sec

In [10]:
y, sr = librosa.load(path, sr=16000)
y.shape

(2468352,)

In [3]:
path = test_sample7_path
sample = load_and_resample(path, target_sr)
RING = CLSforRing('RF')
s = 0
sample_sec = extract_sec(sample, target_sr, start = int(s/target_sr))

In [11]:
import librosa


In [17]:
sample_rate = 8000
min_duration = 1.0
frame_length = 8064
hop_length_frame = 8064
n_fft = 255
hop_length_fft = 63
sequence_sample_length = librosa_waveform.shape[0]

waveform_np = sample_sec.squeeze().numpy()
# librosa로 처리 (sample_rate와 함께 사용 가능)
librosa_waveform = librosa.resample(waveform_np, orig_sr=16000, target_sr=sample_rate) 
sound_data_list = [librosa_waveform[start:start + frame_length] for start in range(
0, sequence_sample_length - frame_length + 1, hop_length_frame)]  # get sliding windows
audio = np.vstack(sound_data_list)
dim_square_spec = int(n_fft / 2) + 1
m_amp_db_audio,  m_pha_audio = numpy_audio_to_matrix_spectrogram(
        audio, dim_square_spec, n_fft, hop_length_fft)

In [3]:
from prediction_denoise import prediction

path = test_sample7_path
# wav 파일 불러오기
sample = load_and_resample(path, target_sr)

weights_path = '/home/ubuntu/jupyter/minyoung/skt/Speech-enhancement/weights/model_weights.pth'
name_model = 'model_unet'
audio_dir_prediction = '/home/ubuntu/jupyter/minyoung/skt/Speech-enhancement/demo_data/test'
dir_save_prediction = '/home/ubuntu/jupyter/minyoung/skt/Speech-enhancement/demo_data/test/sample_denoise/'
audio_input_prediction = [path]
audio_output_prediction = 'sample32_denoise.wav'
sample_rate = 8000
min_duration = 1.0
frame_length = 8064
hop_length_frame = 8064
n_fft = 255
hop_length_fft = 63

# Prediction 및 enhancement 단계
prediction(weights_path, name_model, audio_dir_prediction, dir_save_prediction,
            audio_input_prediction, audio_output_prediction, sample_rate, min_duration,
            frame_length, hop_length_frame, n_fft, hop_length_fft)

# 링백톤 분류기 불러오기
RING = CLSforRing('RF')

# 1초 단위로 예측
for s in range(0, sample.shape[-1]- target_sr, target_sr):
    sample_sec = extract_sec(sample, target_sr, start = int(s/target_sr))
    sample_filter = preprocess(sample_sec, n_fft, hop_length, hann_window, target_sr, config['freq_range']).float()
    is_sys = RING.cls(sample_filter.reshape(1,-1))
    if is_sys == 'no sys':
        with torch.no_grad():
            out_dict = model(forpred(sample_filter.unsqueeze(0)).unsqueeze(0).type(torch.FloatTensor).transpose(2,3).to(device))
        test_pred=out_dict['event_logit']
        no_speech = torch.argmax(test_pred, dim = -1).item()
        if no_speech:
            print(f"{int(s/target_sr)} : {int(s/target_sr)+1} - music") 
        else:
            #######################
            ### Enhancement
            #######################
            # 경로 및 파라미터들 직접 작성
            

            print(f"{int(s/target_sr)} : {int(s/target_sr)+1} - speech")

    else:
        print(f"{int(s/target_sr)} : {int(s/target_sr)+1} - {is_sys}") 

/tmp/ipykernel_584830/2614224937.py:6: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at ../aten/src/ATen/native/Copy.cpp:305.)
  sample_filter = preprocess(sample_sec, n_fft, hop_length, hann_window, target_sr, config['freq_range']).float()


0 : 1 - INTL_TERM_ANM_wb.wav
1 : 2 - INTL_TERM_ANM_wb.wav
2 : 3 - INTL_TERM_ANM_wb.wav
3 : 4 - speech
4 : 5 - speech
5 : 6 - speech
6 : 7 - speech
7 : 8 - speech
8 : 9 - speech
9 : 10 - speech
10 : 11 - speech
11 : 12 - speech
12 : 13 - speech
13 : 14 - speech
14 : 15 - speech
15 : 16 - speech
16 : 17 - speech
17 : 18 - speech
18 : 19 - speech
19 : 20 - speech
20 : 21 - speech
21 : 22 - speech
22 : 23 - speech
23 : 24 - speech
24 : 25 - speech
25 : 26 - speech
26 : 27 - speech
27 : 28 - speech
28 : 29 - speech
29 : 30 - speech
30 : 31 - speech
31 : 32 - speech
32 : 33 - speech
33 : 34 - speech
34 : 35 - speech
35 : 36 - speech
36 : 37 - speech
37 : 38 - speech
38 : 39 - speech
39 : 40 - speech
40 : 41 - speech
41 : 42 - speech
42 : 43 - speech
43 : 44 - speech
44 : 45 - speech
45 : 46 - speech
46 : 47 - speech
47 : 48 - speech
48 : 49 - speech
49 : 50 - speech
50 : 51 - speech
51 : 52 - speech
52 : 53 - speech
53 : 54 - speech
54 : 55 - speech
55 : 56 - speech
56 : 57 - speech
57 : 58 